In [ ]:
!git clone https://github.com/mueedurrehman/distiller.git
%cd distiller
!pip3 install -e .

Cloning into 'distiller'...
remote: Enumerating objects: 6174, done.
remote: Total 6174 (delta 0), reused 0 (delta 0), pack-reused 6174
Receiving objects: 100% (6174/6174), 39.82 MiB | 30.68 MiB/s, done.
Resolving deltas: 100% (4331/4331), done.
/content/distiller
Obtaining file:///content/distiller
     |████████████████████████████████| 2.1MB 7.1MB/s 
     |████████████████████████████████| 734.6MB 24kB/s 
     |████████████████████████████████| 10.2MB 24.8MB/s 
     |████████████████████████████████| 460kB 56.6MB/s 
     |████████████████████████████████| 110.5MB 63kB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 798kB 52.0MB/s 
     |████████████████████████████████| 112kB 57.1MB/s 
     |████████████████████████████████| 1.7MB 55.8MB/s 
     |████████████████████████████████| 235kB 58.0MB/s 
     |████████████████████████████████| 153kB 59.0MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |█████████████████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time
import math
import yaml
import distiller
from collections import OrderedDict
import os
import pandas as pd
import distiller.models
import distiller.quantization
from collections import OrderedDict
import copy

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Pruning Schedules/OneShotElementBaseNoOptResults/92.49-82.6/best.pth.tar" .
# !cp ../../../drive/MyDrive/CS6787\ -\ Final\ Project/Google\ Colab\ Scripts/Quantization\ Models/Resnet56\ Quantization\ Stats/resnet56_quant_stats.yaml .

OneShotElementBaseNoOptResults/92.49-82.6

92.49
82.6

In [ ]:
!python3 /content/distiller/examples/classifier_compression/compress_classifier.py -a resnet56_cifar --compress="/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Quantization Schedules/QA/Asymmetric/4bitAsymmetricUQA.yaml" ./data --resume="best.pth.tar" -o "/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Quantization Schedules/SparseQAAsymmetric" --qe-stats-file="/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Quantization Schedules/acts_quantization_stats.yaml" --qe-clip-acts=avg --epochs 25 --vs=0 --lr=0.0001 --prioritizeAccuracy

Streaming output truncated to the last 5000 lines.
| 170 | module.layer3.1.conv2.weight_scale             | (64, 1, 1, 1)  |            64 |             64 |    0.00000 |    0.00000 |  0.00000 |  0.00000 |  0.00000 |    0.00000 | 21.63986 |  48.04601 |   48.04601 |
| 171 | module.layer3.1.conv2.weight_zero_point        | (64, 1, 1, 1)  |            64 |             62 |    0.00000 |    0.00000 |  0.00000 |  3.12500 |  3.12500 |    3.12500 |  2.18030 |  -7.23438 |    7.23438 |
| 172 | module.layer3.2.conv1.float_weight             | (64, 64, 3, 3) |         36864 |           3687 |    0.00000 |    0.00000 |  7.81250 | 56.64062 |  0.00000 |   89.99837 |  0.03215 |  -0.00030 |    0.00955 |
| 173 | module.layer3.2.conv1.weight                   | (64, 64, 3, 3) |         36864 |           3672 |    0.00000 |    0.00000 |  7.81250 | 56.68945 |  0.00000 |   90.03906 |  0.03223 |  -0.00031 |    0.00955 |
| 174 | module.layer3.2.conv1.weight_scale             | (64, 1, 1, 1)  |            64 |

In [ ]:
!python3 /content/distiller/examples/classifier_compression/compress_classifier.py -a resnet56_cifar --compress="/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Quantization Schedules/QA/Asymmetric/4bitAsymmetricUQA.yaml" ./data --resume="best.pth.tar" -o "/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Quantization Schedules/SparseQAAsymmetric" --qe-stats-file="/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Quantization Schedules/acts_quantization_stats.yaml" --qe-clip-acts=none --epochs 25 --vs=0 --lr=0.0001 --prioritizeAccuracy

Streaming output truncated to the last 5000 lines.
| 170 | module.layer3.1.conv2.weight_scale             | (64, 1, 1, 1)  |            64 |             64 |    0.00000 |    0.00000 |  0.00000 |  0.00000 |  0.00000 |    0.00000 | 21.68624 |  48.06691 |   48.06691 |
| 171 | module.layer3.1.conv2.weight_zero_point        | (64, 1, 1, 1)  |            64 |             62 |    0.00000 |    0.00000 |  0.00000 |  3.12500 |  3.12500 |    3.12500 |  2.18030 |  -7.23438 |    7.23438 |
| 172 | module.layer3.2.conv1.float_weight             | (64, 64, 3, 3) |         36864 |           3687 |    0.00000 |    0.00000 |  7.81250 | 56.64062 |  0.00000 |   89.99837 |  0.03215 |  -0.00030 |    0.00955 |
| 173 | module.layer3.2.conv1.weight                   | (64, 64, 3, 3) |         36864 |           3671 |    0.00000 |    0.00000 |  7.81250 | 56.68945 |  0.00000 |   90.04178 |  0.03223 |  -0.00030 |    0.00955 |
| 174 | module.layer3.2.conv1.weight_scale             | (64, 1, 1, 1)  |            64 |